In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from utilities import read_dataset_folder, describe, statistics

import random
import numpy as np
import ast

class Collector(ast.NodeVisitor):
    def __init__(self):
        self.nodes = []
    
    def visit(self, node):
        name = type(node).__name__
        self.nodes.append(name)
        super().visit(node)

def vectorize(codes):
    nodes = set()
    nsets = []
    for code in codes:
        tree = ast.parse(code)
        visitor = Collector()
        visitor.visit(tree)
        nodes.update(visitor.nodes)
        nsets.append(visitor.nodes)
    nodes = list(nodes)
    freqs = {node: sum(node in nset for nset in nsets) for node in nodes}
    print(f'AST hist dim: {len(nodes)}')
    vecs = []
    for nset in nsets:
        total = len(nset)
        vec = []
        for node in nodes:
            tf = len([i for i in nset if node == i]) / total
            idf = np.log(len(nsets) / freqs[node])
            vec.append(tf * idf)
        vecs.append(vec)
    return vecs

def dataset(folder='data', count=2000):
    random.seed(42)
    programs = random.sample(read_dataset_folder(folder), count)
    print(f'Total programs: {len(programs)}')
    vectors = vectorize([code for code, _ in programs])
    x = np.array(list(vectors), dtype=np.float32)
    y = np.array([klass for _, klass in programs], dtype=int).reshape((len(programs), 1))
    return None, x, y

describe(dataset)

Total programs: 100
AST hist dim: 75
Components: 75.
Non-zero components: 11.
Zero components: 64.


**Plots 100-1000**

In [2]:
stats = statistics(dataset, KNeighborsClassifier, {
    'n_neighbors': [4],
    'weights': ('distance',),
    'metric': ('manhattan',)
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s

{'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}
{'accuracy': (0.85, 0.084),
 'f1_macro': (0.817, 0.088),
 'f1_weighted': (0.842, 0.08),
 'fit_time': (0.0, 0.0),
 'precision_macro': (0.848, 0.075),
 'precision_weighted': (0.888, 0.05),
 'recall_macro': (0.842, 0.075),
 'recall_weighted': (0.85, 0.084),
 'score_time': (0.003, 0.0)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4, weights=d

In [3]:
stats = statistics(dataset, SVC, {
    'C': [30],
    'kernel': ('rbf',),
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s

{'C': 30, 'kernel': 'rbf'}
{'accuracy': (0.82, 0.051),
 'f1_macro': (0.787, 0.05),
 'f1_weighted': (0.814, 0.047),
 'fit_time': (0.001, 0.0),
 'precision_macro': (0.819, 0.055),
 'precision_weighted': (0.858, 0.045),
 'recall_macro': (0.807, 0.05),
 'recall_weighted': (0.82, 0.051),
 'score_time': (0.003, 0.0)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END

In [4]:
stats = statistics(dataset, RandomForestClassifier, {
    'n_estimators': [300],
    'max_depth': [40],
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s

{'max_depth': 40, 'n_estimators': 300}
{'accuracy': (0.91, 0.08),
 'f1_macro': (0.886, 0.11),
 'f1_weighted': (0.897, 0.094),
 'fit_time': (0.183, 0.004),
 'precision_macro': (0.896, 0.11),
 'precision_weighted': (0.912, 0.091),
 'recall_macro': (0.902, 0.093),
 'recall_weighted': (0.91, 0.08),
 'score_time': (0.016, 0.001)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; total time=  

In [5]:
stats = statistics(dataset, MLPClassifier, {
    'activation': ['relu'],
    'learning_rate_init': [0.001],
}, 100, 1000 + 100, 100)

Total programs: 100
AST hist dim: 75
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s

{'activation': 'relu', 'learning_rate_init': 0.001}
{'accuracy': (0.46, 0.073),
 'f1_macro': (0.32, 0.094),
 'f1_weighted': (0.344, 0.087),
 'fit_time': (0.056, 0.003),
 'precision_macro': (0.296, 0.113),
 'precision_weighted': (0.317, 0.11),
 'recall_macro': (0.415, 0.081),
 'recall_weighted': (0.46, 0.073),
 'score_time': (0.002, 0.0)}
Total programs: 200
AST hist dim: 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate_init=0.001; 